Cell below loads all libraries used within this project.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import os
import gzip
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal
import itertools
from scipy import linalg
import matplotlib as mpl

This method was used from within the utils folder of the dataset to load different datasets and images withing fashion-MNIST dataset.

In [ ]:
def load_mnist(path, kind='train'):
    
    ''' from https://github.com/zalandoresearch/fashion-mnist'''

    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

Principle components analysis or PCA, is a technique for reducing the dimensionality of datasets. The aim is to increase a databases interpretability while minimizing information loss. This is achieved by creating new uncorrelated variables that successively maximize variance. in the cell below, the dataset is loaded and then standardized by dividing them by 225 since they are all pixel values. Furthermore, their mean is calculated, and the data is normalized to make the PCA appliance easier. The first 25 images are shown with matching labels in the block below. Method 'conv_2d' splits the 1d input of 784 pixels into 2d with the shape (28,28) to make the plotting possible.

There are 10 unique classes within this dataset which are : 0-t-shirt/top', 1-'Trouser', 2-'Pullover', 3-'Dress', 4-'Coat', 5-'Sandal', 6-'Shirt', 7-'Sneaker', 8-'bag', 9-'ankle boot.

In [ ]:
fashion_X_train, train_y_labels = load_mnist('C:/Uni/Year3/MLCW//fashion-mnist/data/fashion', kind='train')
fashion_X_test, test_y_labels = load_mnist('C:/Uni/Year3/MLCW/fashion-mnist/data/fashion', kind='t10k')

class_names = ['t-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'bag', 'ankle boot']

#standardization: devided by 255
x_train_standard = fashion_X_train/255
x_test_standard = fashion_X_test/255

#normalizing by finding mean and calculating distance from mean
train_mean = np.mean(x_train_standard)
test_mean = np.mean(x_test_standard)

train_normal = x_train_standard - train_mean
test_normal = x_test_standard - test_mean


def conv_2d(matrix):
    for x in matrix:
        matrix = np.reshape(matrix, (28,28))
    return matrix

plt.figure(figsize=(7,7))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    train_images = conv_2d(fashion_X_train[i])
    plt.imshow(train_images, cmap=plt.cm.binary)
    plt.xlabel(class_names[train_y_labels[i]])
plt.show()

The dimensionality of the data is 784, consistant of 28x28 pixels. PCA is performed here to understand whether this reduction significantly impacts our information retention or not. a good reduction will aim to keep most of the information while discarding as many dimensions as possible. Here, we learn some quantities about these data namely different components and cumulative variance of the components. These represent the principal components of the data. in the code cell below evector of >=0.8 is taken from the cumulative variance as we want to retain 80+% of our datapoints as it is the ideal number. This is done to gain an understanding of what would be an ideal reduction.

In [ ]:
pca = PCA()
fit_all = pca.fit_transform(train_normal)

cumilative = np.cumsum(pca.explained_variance_)


explained_var = pca.explained_variance_ratio_
cumulative_ratio = np.cumsum(explained_var)
less_eq_80 = np.where(cumulative_ratio >= 0.8)[0][0] #evector with cumulative variance ratio of >=0.8

# Plot cumulative explained variance ratio
pcs = list(range(1, len(explained_var)+1)) #list of all principle components
plt.plot(pcs, np.cumsum(explained_var), color= 'black')
plt.ylabel('Cumulative Variance Ratio')
plt.xlabel('Principal Component')
plt.show()

print("ideal pc retention = "+str(less_eq_80+1))
print(f"first 5 PCs: {pca.explained_variance_[:10]}")

PCA was applied to all data and the pca object of the first 10 principles were returned. With this analysis we intend to keep higher variances. The output tells us that the cumalitive variance decreases rapidly between first 5 components. Furthermore, to understand how many dimension would be ideal to retain, Cumulative Variance Ratio per Principal Component is plotted. This better visualizes the pca of whole data. It can be observed that the first "24" principle components can explain 80% of the variance in this dataset. so reducing the datasets to 24 dimensions is ideal.

in the cell below we reduce the dimensionality of our data to 2 components. the data is then visualized as we plot the first component against the second component. Cumulative variance of the two components are also printed.

In [ ]:
pca_2 = PCA(n_components=2)
fit_2d = pca_2.fit_transform(train_normal)
cum_var = pca_2.explained_variance_#variance
cumalitive_var = np.cumsum(cum_var)#cumulative var
print(cumalitive_var)


plt.figure(figsize=(10,10))
scatter = plt.scatter(fit_2d.T[0], fit_2d.T[1], c=train_y_labels)
clb = plt.colorbar(scatter) #colour coding
clb.ax.set_title('Class') #colour coding
plt.xlabel("1st principle comp")
plt.ylabel("2nd principle comp")
plt.title('2D pca')

The first principle component is "19.80980567" and the second principle component is "31.92201614" clearly showing the gap between the two components are high. This tells us that reducing to 2d from 780+ dimensions is not a good idea since all the classes keep clustering into each other and there is no clear separation between the classes and most of the datapoint are lost and we are underfitting our data. We can furthur prove that this reduction is not fitting with plotting our images with this reduction

In [ ]:
# Plot the grid of images
fig, axes = plt.subplots(2,2, figsize = (10,5), sharex=True, sharey = True, constrained_layout = True )

# First 2 principal components
f_2_PCS = [pca.components_[i].reshape(28,28) for i in range(0,2)]

for pc in range(0,2):
    axes[0,pc].imshow(f_2_PCS[0], aspect='auto', cmap = "gray_r")
    axes[1,pc].imshow(f_2_PCS[pc], aspect='auto', cmap = "gray_r")
plt.show()

It is demonstrated that the images come out extremly blurry and do not really represeant all of the shapes we will see within our dataset. Moreover, these images show that most variance in the data is found between the pixel values of a dark shirt and the pixel values of a white shoe.

in the code block below a simple K-means clustering is being employed. Each data is being assigned a cluster and cluster centres are calculated and marked inside each cluster. This is an unsupervised task with the task to separate points inside a dataset into different clusters such that elements within each cluster are similar. GMM is an example of soft clustering while K-mean is an example of hard clustering

In [ ]:
kmeans = KMeans(n_clusters=10)
label = kmeans.fit_predict(fit_2d)
centroid = kmeans.cluster_centers_
unique = np.unique(label)

plt.figure(figsize=(10,10))
for i in unique:
    plt.scatter(fit_2d[label == i , 0] , fit_2d[label == i , 1] , label = i)
    plt.scatter(centroid[:,0], centroid[:,1], s = 80, color = 'b')
plt.show()

GMM

Gaussian mixture model is a probabilistic model that is assembled by different gaussian distributions with each having their own mean and standard deviation. in GMM, we do a weighted average of all of them to mix them into one gaussian distribution hence the name mixture model. Since GMM is a probabilistic model, it is possible to find probabilistic cluster assignments. Using "fit_predict()" to find clusters for a number of our data points, 300 to be exact. Predictions are then scattered into a plot for better visualization. 
Furthermore, the cluster centroids are represented by a grey X marker within the scatters. the cluster centres are 
[[ 4.09073185  1.54469529], [-6.22776316  0.79198387], [-0.28301486 -5.53351972], [ 2.73385548 -4.09994746], [ 0.05194298  5.07584905], [ 0.70442081 -0.45591165], [-3.93130757  3.35630958],
[ 6.96162988  2.70717704], [-3.02978303 -2.5079944 ], [ 5.79741477 -2.17292458]].
z-score for this model is "-5.016039953087466" with some minor overlapping happening between the classes. Keeping in mind only 3000 sample data were used for optimization. This score reveals that these datapoints are about 5 standard deviations away from the mean. Data's standard deviation is "4.021403225610568" and the mean is "0.29961496697739776". Since there is a large gap between the mean and standard deviation, it suggests that our data are scattered and not close to each other. It is worth mentioning that the random states in the GMM below is set to zero hence making it possible that the EM step or expectation maximization step, has missed globally optimal solutions.

In [ ]:
colours = ['red', 'green', 'purple', 'yellow', 'blue', 'cyan', 'black', 'grey', 'orange', 'brown']

gmm = GaussianMixture(n_components=10, covariance_type='full', random_state=0)
x = fit_2d[:3000,:3000]
mea = gmm.fit(x).means_
means = np.mean(mea)
labels = gmm.fit_predict(x)
score = gmm.score(x)
std = np.std(x)
print('std', std)
print('mean', means)
print('score', score)

centers = np.empty(shape=(gmm.n_components, x.shape[1]))
mean_centre = gmm.means_
cov_centre = gmm.covariances_
plt.figure(figsize=(10,10))
for i in range(gmm.n_components):
    density = multivariate_normal(mean_centre[i], cov_centre[i]).logpdf(x)
    centers[i, :] = x[np.argmax(density)]
plt.scatter(centers[:, 0], centers[:, 1], s=500, c='grey', marker='X')
pred = plt.scatter(x[:, 0], x[:, 1], c=labels, cmap='viridis', s=20) #predictions
clb = plt.colorbar(pred) #colour coding
clb.ax.set_title('Class labels') #colour coding
plt.show()
print(centers)

Below we manipulate the data furthure by changing the number of random states present within our model to 42 for understanding whether this will have an impact or not. This vatiable, changes the test, train split, furthure splliting our 3000 data points into test and train subsets. For consistancy, this value is used since not specifying it will lead to different splits and diffrent z score each time GMM is applied.

In [ ]:
gmm = GaussianMixture(n_components=10, covariance_type='full', random_state=42)
x = fit_2d[:3000,:3000]
mea = gmm.fit(x).means_
means = np.mean(mea)
labels = gmm.fit_predict(x)
score = gmm.score(x)
std = np.std(x)
print('std', std)
print('mean', means)
print('score', score)

centers = np.empty(shape=(gmm.n_components, x.shape[1]))
mean_centre = gmm.means_
cov_centre = gmm.covariances_
plt.figure(figsize=(10,10))
for i in range(gmm.n_components):
    density = multivariate_normal(mean_centre[i], cov_centre[i]).logpdf(x)
    centers[i, :] = x[np.argmax(density)]
plt.scatter(centers[:, 0], centers[:, 1], s=500, c='grey', marker='X')
pred = plt.scatter(x[:, 0], x[:, 1], c=labels, cmap='viridis', s=20) #predictions
clb = plt.colorbar(pred) #colour coding
clb.ax.set_title('Class labels') #colour coding
plt.show()
print(centers)

By changing the random state, the standard deviation of data remains the same while the mean increases. The change in the mean impacts our Z score as well but since the change in our mean is minimal, this does not change anything. The cluster centres do change which is the expected observation however, the genral vicinity of them do not different from when random_state was set to 0.

The cluster centroid are consistant with the K-means clustering. It remains to be seen wheteher it is consitant when all dimensions are present. As shown by 2d reduction in PCA, majority of the data is lost so it is expected for these clusters to not be accurate and not a good representation of different classes.